# 그래프 시각화

In [ ]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.graph_objects as go
import plotly.express as px

# 모듈 모음
import modules as mds

In [ ]:
# Windows에서 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕'이 설치되어 있을 경우
plt.rcParams['axes.unicode_minus'] = False     # 마이너스(-) 부호 깨짐 방지

In [ ]:
# macOS에서 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'   # macOS 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 연도별 신규 발전량 평균 성장 데이터 그래프 제작

# 연도별 신규 발전량 데이터 저장
gen_new_total = pd.read_excel("dataset/지역별_연도별_신규_발전량.xlsx")

# 연도별 모든 지역의 평균값 계산 후 새로운 df 생성
area_new_col = gen_new_total.columns[1:-1]
year_new_mean = gen_new_total[area_new_col].mean(axis=1).round().astype(int)
year_new_mean_df = pd.DataFrame({"연도": gen_new_total["연도"], "평균 발전량(MWh)": year_new_mean})

# 그래프 생성
year_new_mean_fig = px.bar(
    x=year_new_mean_df["연도"],
    y=year_new_mean_df["평균 발전량(MWh)"],
    color=year_new_mean_df["평균 발전량(MWh)"],
    color_continuous_scale=["#fff8e1", "#ffcc80", "#ff8a65"],
    text=[f"{i:,.0f}" for i in year_new_mean_df["평균 발전량(MWh)"]]
    )

# 막대 넓이 설정
year_new_mean_fig.update_traces(width=0.4, textfont=dict(size=15))

# 테두리 설정
year_new_mean_fig.update_traces(
    width=0.4,
    # marker_line=dict(color='black', width=1),
    textposition='outside'
    )

# 타이틀 설정
year_new_mean_fig.update_layout(
    title="연도별 평균 발전량 추이",
    template="plotly_white"
    )

# 그리드 설정
year_new_mean_fig.update_layout(
    xaxis=dict(title="연도", dtick=1, showgrid=False),
    yaxis=dict(title="평균 발전량(MWh)", showgrid=True, gridcolor="rgba(211, 211, 211, 0.8)")
    )

# 크기 설정
year_new_mean_fig.update_layout(width=1440,height=1080, font=dict(size=20))

year_new_mean_fig.show()

In [ ]:
# 경북 / 경남 / 충남 그래프

merged = pd.read_excel("dataset/최종일사량합친데이터.xlsx")

# === 지역 필터링 & 정리 ===
target_regions = ["경북", "경남", "충남"]
df_filtered = merged[merged["지역"].isin(target_regions)].copy()

df_filtered["연도"] = pd.to_numeric(df_filtered["연도"], errors="coerce")
df_grouped = (
    df_filtered.groupby(["연도", "지역"])[["발전량(MWh)", "설비용량(MW)"]]
    .sum()
    .reset_index()
)

# === 색상 고정 ===
color_map = {
    "경남": "#1f77b4",  # 파랑
    "경북": "#d62728",  # 빨강
    "충남": "#2ca02c"   # 초록
}

# ====================================================
# 1) 연도별 발전량 추이 (라인 차트)
# ====================================================
fig_gen = go.Figure()

for region in target_regions:
    df_region = df_grouped[df_grouped["지역"] == region]
    fig_gen.add_trace(
        go.Scatter(
            x=df_region["연도"],
            y=df_region["발전량(MWh)"],
            mode="lines+markers",
            name=region,
            line=dict(color=color_map[region], width=3),
            marker=dict(size=9, symbol="circle")
        )
    )

fig_gen.update_layout(
    title="경북·경남·충남 연도별 발전량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="발전량(MWh)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)",
                bordercolor="lightgray", borderwidth=1)
)

# 크기 설정
fig_gen.update_layout(width=1440,height=1080, font=dict(size=20))

fig_gen.show()

# ====================================================
# 2) 연도별 설비용량 추이 (라인 차트)
# ====================================================
fig_cap = go.Figure()

for region in target_regions:
    df_region = df_grouped[df_grouped["지역"] == region]
    fig_cap.add_trace(
        go.Scatter(
            x=df_region["연도"],
            y=df_region["설비용량(MW)"],
            mode="lines+markers",
            name=region,
            line=dict(color=color_map[region], width=3, dash="dot"),  # 점선
            marker=dict(size=8, symbol="square")
        )
    )

fig_cap.update_layout(
    title="경북·경남·충남 연도별 설비용량 추이",
    template="plotly_white",
    xaxis=dict(title="연도", dtick=1, showgrid=True, gridcolor="lightgray"),
    yaxis=dict(title="설비용량(MW)", showgrid=True, gridcolor="lightgray"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)",
                bordercolor="lightgray", borderwidth=1)
)

# 크기 설정
fig_cap.update_layout(width=1440,height=1080, font=dict(size=20))

fig_cap.show()

In [ ]:
# 3지역 10년간 일사량 추이 그래프

# === 데이터 불러오기 ===
df = pd.read_excel("dataset/2015-2024_충남,경북,경남_평균_일사량.xlsx")

# === Wide → Long 변환 ===
df_long = df.melt(id_vars="지역", var_name="연도", value_name="일사량(MJ/m2)")
df_long["연도"] = pd.to_numeric(df_long["연도"], errors="coerce")

# === Plotly 선 그래프 ===
fig = px.line(
    df_long,
    x="연도",
    y="일사량(MJ/m2)",
    color="지역",
    markers=True,
    title="경북·경남·충남 2015~2024 평균 일사량 추이",
    category_orders={"지역": ["경북", "경남", "충남"]},  # 순서 강제
    color_discrete_map={
        "경남": "#1f77b4",  # 파랑
        "경북": "#d62728",  # 빨강
        "충남": "#2ca02c"   # 초록
    }
)

# 스타일 조정
fig.update_traces(line=dict(width=3), marker=dict(size=9))
fig.update_layout(
    template="plotly_white",
    xaxis=dict(dtick=1, title="연도"),
    yaxis=dict(title="일사량 (MJ/m²)"),
    legend=dict(title="지역", bgcolor="rgba(255,255,255,0.7)", bordercolor="lightgray", borderwidth=1)
)

# 크기 설정
fig.update_layout(width=1440,height=1080, font=dict(size=20))

fig.show()

In [ ]:
# 지역별 발전량 / 설비용량 그래프

# 데이터 로드
df = pd.read_excel("dataset/일사량합친데이터.xlsx")

# 2024년만 선택
df_2024 = df[df["연도"] == 2024].copy()

# 발전량 그래프
fig_gen = go.Figure()
fig_gen.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["발전량(MWh)"],
    text=[f"{v:,.0f}" for v in df_2024["발전량(MWh)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["발전량(MWh)"],
        colorscale="Blues",
        showscale=True,
        colorbar=dict(title="MWh")
    ),
    hovertemplate="<b>%{x}</b><br>발전량: %{y:,.0f} MWh<extra></extra>"
))
fig_gen.update_layout(
    title="2024년 지역별 발전량",
    xaxis_title="지역",
    yaxis_title="발전량 (MWh)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_gen.update_xaxes(tickangle=-45)
fig_gen.update_yaxes(tickformat=",.0f")

# 테두리 설정
# fig_gen.update_traces(marker_line=dict(color='black', width=1))

# 크기 설정
fig_gen.update_layout(width=1440,height=1080, font=dict(size=20))

# 결과 출력
fig_gen.show()

# 설비용량 그래프
fig_cap = go.Figure()
fig_cap.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["설비용량(MW)"],
    text=[f"{v:,.0f}" for v in df_2024["설비용량(MW)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["설비용량(MW)"],
        colorscale="Greens",
        showscale=True,
        colorbar=dict(title="MW")
    ),
    hovertemplate="<b>%{x}</b><br>설비용량: %{y:,.0f} MW<extra></extra>"
))
fig_cap.update_layout(
    title="2024년 지역별 설비용량",
    xaxis_title="지역",
    yaxis_title="설비용량 (MW)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_cap.update_xaxes(tickangle=-45)
fig_cap.update_yaxes(tickformat=",.0f")

# 테두리 설정
# fig_cap.update_traces(marker_line=dict(color='black', width=1))

# 크기 설정
fig_cap.update_layout(width=1440,height=1080, font=dict(size=20))

# 결과 출력
fig_cap.show()

In [ ]:
# 2024 지역별 일사량 그래프

# 2024년도 데이터만 선택
df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 막대그래프
fig = px.bar(
    df_2024,
    x="지역",
    y="일사량(MJ/m2)",
    text="일사량(MJ/m2)",
    color="일사량(MJ/m2)",   # 값에 따라 색상 구분
    color_continuous_scale="YlOrRd",  # 노랑~빨강 색상 스케일
    title="2024년 지역별 일사량 (MJ/m²)"
)

# 레이블, 서식 조정
fig.update_traces(texttemplate="%{text:.1f}", textposition="outside")
fig.update_layout(
    xaxis_title="지역",
    yaxis_title="일사량 (MJ/m²)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=80)
)
fig.update_xaxes(tickangle=-45)

# 크기 설정
fig.update_layout(width=1440,height=1080, font=dict(size=20))

fig.show()

In [ ]:
# 2024 지역별 설비이용률 그래프

# 데이터 로드
df = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df[df["연도"] == 2024].copy()

# 설비이용률 그래프
fig_cf = go.Figure()
fig_cf.add_trace(go.Bar(
    x=df_2024["지역"],
    y=df_2024["설비이용률(%)"],
    text=[f"{v:.2f}%" for v in df_2024["설비이용률(%)"]],
    textposition="outside",
    marker=dict(
        color=df_2024["설비이용률(%)"],
        colorscale="Oranges",
        showscale=True,
        colorbar=dict(title="설비이용률 (%)")
    ),
    hovertemplate="<b>%{x}</b><br>설비이용률: %{y:.2f}%<extra></extra>"
))

fig_cf.update_layout(
    title="2024년 지역별 설비이용률",
    xaxis_title="지역",
    yaxis_title="설비이용률 (%)",
    template="plotly_white",
    margin=dict(l=40, r=40, t=60, b=100)
)
fig_cf.update_xaxes(tickangle=-45)
fig_cf.update_yaxes(tickformat=".2f")

# 크기 설정
fig_cf.update_layout(width=1440,height=1080, font=dict(size=20))

fig_cf.show()
